# Introduction
In this notebook we will write out what happens behind the scene, when we in pytorch write **loss.backward()**. This is an important [exercise](https://karpathy.medium.com/yes-you-should-understand-backprop-e2f06eab496b), as it will make us much more effective at building and debugging neural networks, by removing "the magic" happening behind the scenes.

# Libraries

In [10]:
%matplotlib inline
%config IPCompleter.use_jedi=False

In [11]:
import random
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

# Data
We are using the same data again again!

### Reading the Data

In [12]:
# Reading names into a list
with open('../../data/names.txt', 'r') as f:
    names = f.readlines()
    names = [name.strip() for name in names]

### Creating Vocabulary

In [13]:
# Building the vocabulary (character to/from index)
chars = sorted(list(set(''.join(names))))
chr_to_idx = {s:i+1 for i,s in enumerate(chars)}; print(chr_to_idx)
chr_to_idx['.'] = 0
idx_to_chr = {i:s for s,i in chr_to_idx.items()}; print(idx_to_chr)

{'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26}
{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


### Building the Dataset

In [14]:
def build_dataset(words: list, block_size: int = 3, verbose: bool = False) -> tuple:
    """
    Description:
        Take a list of words and creates Xs and Ys give
        a block size. E.g. block_size=3 for emma would 
        result in:
            ... ---> e  
            ..e ---> m  
            .em ---> m  
            emm ---> a  
            mma ---> . 
        Stored in a (n, 3) tensor
    Inputs:
        words: List of words
        block_size: The context window
        verbose: Where to print the outputs
    Ouputs:
        A (n, block_size) tensor with Xs
        A (n) tensor with Ys
    """
    X, Y = [], []
    for word in words:
        if verbose:
            print(word)
        context = [0] * block_size
        for char in word + '.':
            idx = chr_to_idx[char]
            X.append(context)
            Y.append(idx)
            if verbose:
                print(''.join(idx_to_chr[i] for i in context), idx_to_chr[idx])
            context = context[1:] + [idx]
    
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    
    return X, Y

In [15]:
# Putting names in random order
random.shuffle(names)

# Creating split indices
n1 = int(0.8 * len(names))
n2 = int(0.9 * len(names))

In [16]:
# Building training, validation and test sets
Xtr, Ytr = build_dataset(names[:n1]); print(Ytr.size())
Xval, Yval = build_dataset(names[n1:n2]); print(Yval.size())
Xtest, Ytest = build_dataset(names[n2:]); print(Ytest.size())

torch.Size([182628])
torch.Size([22776])
torch.Size([22742])


# Utility
Here are some utility functions for various purposes later.

In [17]:
def cmp(s: str, dt: torch.Tensor, t: torch.Tensor):
    """
    Description:
        Compares manually calculated gradients to gradients
        calculated using Pytorch. Prints exact gradients, 
        approximate gradients and the biggest difference
        between gradients.
    Inputs:
        s: name
        dt: Manual gradient
        t: Pytorch gradient
    """
    ex = torch.all(dt == t.grad).item()
    app = torch.allclose(dt, t.grad)
    maxdiff = (dt - t.grad).abs().max().item()
    print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

# Network
We will be using one of the networks from the previous lesson that includes the bachnormalization layer.

In [18]:
n_embd = 10
n_hidden = 200
block_size = 3
vocab_size = len(chr_to_idx)

# Lookup table
C = torch.randn([vocab_size, n_embd])

# Weights and biases
## Layer 1
W1 = torch.randn([n_embd * block_size, n_hidden]) * (5/3)/((n_embd * block_size) ** 0.5)
b1 = torch.randn([n_hidden]) * 0.1

## Layer 2
W2 = torch.randn([n_hidden, vocab_size]) * 0.1
b2 = torch.randn([vocab_size]) * 0.1

## Batch Norm Shifting weights
bngain = torch.ones((1, n_hidden)) * 0.1 + 1
bnbias = torch.zeros((1, n_hidden)) * 0.1

# Placing parameters in list
parameters = [C, W1, b1, W2, b2, bngain, bnbias]

# Enabling gradients
for p in parameters:
    p.requires_grad = True
print("Number of parameters: " + str(sum(p.nelement() for p in parameters)))

Number of parameters: 12297


# Training
 The training loop will also be almost the same except that **loss.backward()** will be written manually. One change is that the forward has been chunked such that we can debug loss.backward() on one step at the time.

### Constructing a mini-batch

In [19]:
batch_size = 32
ix = torch.randint(0, Xtr.shape[0], (batch_size,))
Xb, Yb = Xtr[ix], Ytr[ix]

### Chunked Forward Pass

In [20]:
# Embeddings
emb = C[Xb]
embcat = emb.view(emb.shape[0], -1) # concatenate the vectors

# Linear layer
hprebn = embcat @ W1 + b1

# BatchNorm layer
bnmeani = 1/batch_size*hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
bnvar = 1/(batch_size-1)*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n)
bnvar_inv = (bnvar + 1e-5)**-0.5
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias

# Non-linearity
h = torch.tanh(hpreact) # hidden layer

# Linear layer
logits = h @ W2 + b2 # output layer

# Loss (cross entropy loss)
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes # subtract max for numerical stability
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdims=True)
counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(batch_size), Yb].mean()

### Pytorch Backward Pass

In [21]:
for p in parameters:
    p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv,
          norm_logits, logit_maxes, logits, h, hpreact, bnraw,
         bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani,
         embcat, emb]:
    t.retain_grad()
loss.backward()
loss

tensor(3.8859, grad_fn=<NegBackward0>)

# Exercise 1
Here we will backprop through the all of the variables as they are defined in the forward pass above. 

#### Derivative of logprobs
An example of the mean operation (see last calculation in the forward pass).

Calculation:  
loss = -(a + b + c) / 3 <=>  
loss = -1/3\*a + -1/3\*b + -1/3\*c

Derivative:  
dloss/da = -1/3 + 0 + 0 <=>  
dloss/da = -1/3  

As only the predicted letters participate in the loss, the remaning letters have a gradient of zero.

In [22]:
# Initializing with zeroes
dlogprobs = torch.zeros_like(logprobs)

# Setting gradients in letter locations
dlogprobs[range(batch_size), Yb] = -1.0/batch_size

# Checking estimated gradients vs pytorch gradients
cmp('logprobs', dlogprobs, logprobs)

logprobs        | exact: True  | approximate: True  | maxdiff: 0.0


#### Derivative of probs
The calculation can be found in the forward pass.

Calculation:  
logprobs = log(a)  

Derivative:  
dlogprobs/da = 1/a    

Chain Rule:
F(x) = f(g(x)) =>  
F'(x) = f'(g(x)) * g'(x)  

In our case this means that:  
F'(x) = 1/probs * dlogprobs

In [23]:
# Setting gradients
dprobs = 1.0/probs * dlogprobs

# Checking estimated gradients vs pytorch gradients
cmp('probs', dprobs, probs)

probs           | exact: True  | approximate: True  | maxdiff: 0.0


#### Derivative of counts_sum_inv
From here on only the calculation of the derivative will be show here, while the chain rule will be applied in the actual calculations.

Calculation:  
counts_sum_inv = a * b  

Derivative:  
dcounts_sum_inv/da = b    

In [24]:
# Setting gradients
dcounts_sum_inv = (counts * dprobs).sum(1, keepdim=True)

# Checking estimated gradients vs pytorch gradients
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)

counts_sum_inv  | exact: True  | approximate: True  | maxdiff: 0.0


#### Calculating the remaining gradients
We will no longer go through explicit examples, but just show the calculations. How to calculate the different derivatives can be found on [Wolframalpha](https://www.wolframalpha.com/input?i=d%2Fdx+x%5E-1)

In [25]:
# counts (part 1)
dcounts = counts_sum_inv * dprobs

In [26]:
# counts_sum
dcounts_sum = (-counts_sum**-2) * dcounts_sum_inv
cmp('counts_sum', dcounts_sum, counts_sum)

counts_sum      | exact: True  | approximate: True  | maxdiff: 0.0


In [27]:
# counts (part 2)
dcounts += torch.ones_like(counts) * dcounts_sum
cmp('counts', dcounts, counts)

counts          | exact: True  | approximate: True  | maxdiff: 0.0


In [28]:
# norm_logits
dnorm_logits = counts * dcounts
cmp('norm_logits', dnorm_logits, norm_logits)

norm_logits     | exact: True  | approximate: True  | maxdiff: 0.0


In [29]:
# logit_maxes
dlogit_maxes = (-dnorm_logits).sum(1, keepdim=True)
cmp('logit_maxes', dlogit_maxes, logit_maxes)

logit_maxes     | exact: True  | approximate: True  | maxdiff: 0.0


In [30]:
# logits
dlogits = dnorm_logits.clone()
dlogits += F.one_hot(logits.max(1).indices, num_classes=logits.shape[1]) * dlogit_maxes
cmp('logits', dlogits, logits)

logits          | exact: True  | approximate: True  | maxdiff: 0.0


#### The derivative for matrix multiplication
Here we make a small example, which illustrates how to calculate the derivatives.

<pre>
d = a @ b + c

| d11 d12 |       | a11 a12 |     | b11 b12 |    | c1 c2 |
| d21 d22 |   =   | a21 a22 |  @  | b21 b22 |  + | c1 c2 | =>

d11 = a11 * b11 + a12 * b21 + c1
d12 = a11 * b12 + a12 * b22 + c2
d21 = a21 * b11 + a22 * b21 + c1
d22 = a21 * b12 + a22 * b22 + c2
</pre>

When finding the derivative with respect to a, it will be:

<pre>
dL/da11 = dL/dd11 * b11 + dL/dd12 * b12
dL/da12 = dL/dd11 * b21 + dL/dd12 * b22
dL/da21 = dL/dd21 * b11 + dL/dd22 * b12
dL/da22 = dL/dd21 * b21 + dL/dd22 * b22
</pre>

This means that the derivative for d is (if rewritten as a matrix multiplication):

<pre>

          | dL/da11    dL/da12 |   | b11    b21 |
dL/da  =  | dL/da21    dL/da22 | * | b12    b22 | <=>

dL/da = dL/dd @ b^T
</pre>

Doing the same for the derivative with respect to b, you will get

<pre>
dL/db = a^T @ dL/dd 
</pre>

Doing the same for the derivative with respect to c, you will get

<pre>
dL/dc = dL/dd @ sum(0)
</pre>

In [31]:
# h
dh = dlogits @ W2.T
cmp('h', dh, h)

h               | exact: True  | approximate: True  | maxdiff: 0.0


In [32]:
# W2
dW2 = h.T @ dlogits
cmp('W2', dW2, W2)

W2              | exact: True  | approximate: True  | maxdiff: 0.0


In [33]:
# b2
db2 = dlogits.sum(0)
cmp('b2', db2, b2)

b2              | exact: True  | approximate: True  | maxdiff: 0.0


In [34]:
# hpreact
dhpreact = (1.0 - h**2) * dh
cmp('hpreact', dhpreact, hpreact)

hpreact         | exact: False | approximate: True  | maxdiff: 4.656612873077393e-10


In [35]:
# bngain
dbngain = (bnraw * dhpreact).sum(0, keepdim=True)
cmp('bngain', dbngain, bngain)

bngain          | exact: False | approximate: True  | maxdiff: 2.7939677238464355e-09


In [36]:
# bnraw
dbnraw = bngain * dhpreact
cmp('bnraw', dbnraw, bnraw)

bnraw           | exact: False | approximate: True  | maxdiff: 9.313225746154785e-10


In [37]:
# bnbias
dbnbias = dhpreact.sum(0, keepdim=True)
cmp('bnbias', dbnbias, bnbias)

bnbias          | exact: False | approximate: True  | maxdiff: 7.450580596923828e-09


In [38]:
# bndiff (part 1)
dbndiff = bnvar_inv * dbnraw

In [39]:
# bnvar_inv 
dbnvar_inv = (bndiff * dbnraw).sum(0, keepdim=True)
cmp('bnvar_inv', dbnvar_inv, bnvar_inv)

bnvar_inv       | exact: False | approximate: True  | maxdiff: 3.725290298461914e-09


In [40]:
# bnvar
dbnvar = (-0.5*(bnvar + 1e-5)**-1.5) * dbnvar_inv
cmp('bnvar', dbnvar, bnvar)

bnvar           | exact: False | approximate: True  | maxdiff: 2.444721758365631e-09


In [41]:
# bndiff2
dbndiff2 = (1.0/(batch_size-1))*torch.ones_like(bndiff2) * dbnvar
cmp('bndiff2', dbndiff2, bndiff2)

bndiff2         | exact: False | approximate: True  | maxdiff: 7.639755494892597e-11


In [42]:
# bndiff (part 2)
dbndiff += (2*bndiff) * dbndiff2
cmp('bndiff', dbndiff, bndiff)

bndiff          | exact: False | approximate: True  | maxdiff: 9.313225746154785e-10


In [43]:
# hprebn (part 1)
dhprebn = dbndiff.clone()

In [44]:
# bnmeani
dbnmeani = (-dbndiff).sum(0)
cmp('bnmeani', dbnmeani, bnmeani)

bnmeani         | exact: False | approximate: True  | maxdiff: 7.450580596923828e-09


In [45]:
# hprebn (part 2)
dhprebn += 1.0/batch_size * (torch.ones_like(hprebn) * dbnmeani)
cmp('hprebn', dhprebn, hprebn)

hprebn          | exact: False | approximate: True  | maxdiff: 9.313225746154785e-10


In [46]:
# embcat
dembcat = dhprebn @ W1.T
cmp('embcat', dembcat, embcat)

embcat          | exact: False | approximate: True  | maxdiff: 5.587935447692871e-09


In [47]:
# W1
dW1 = embcat.T @ dhprebn
cmp('W1', dW1, W1)

W1              | exact: False | approximate: True  | maxdiff: 9.313225746154785e-09


In [48]:
def cmp(s, dt, t):
  ex = torch.all(dt == t.grad).item()
  app = torch.allclose(dt, t.grad)
  maxdiff = (dt - t.grad).abs().max().item()
  print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

In [49]:
# b1
db1 = dhprebn.sum(0)
cmp('b1', db1, b1)

b1              | exact: False | approximate: True  | maxdiff: 7.450580596923828e-09


In [50]:
# emb
demb = dembcat.view(emb.shape)
cmp('emb', demb, emb)

emb             | exact: False | approximate: True  | maxdiff: 5.587935447692871e-09


In [51]:
# C
dC = torch.zeros_like(C)
for k in range(Xb.shape[0]):
  for j in range(Xb.shape[1]):
    ix = Xb[k,j]
    dC[ix] += demb[k,j]
cmp('C', dC, C)

C               | exact: False | approximate: True  | maxdiff: 7.450580596923828e-09


# Exercise 2
In exercise 1 we broke down all the operations into atomic pieces, but that is not necessary to do backprop! Here we will do backprop directly on cross_entropy instead of on its small pieces.

In [53]:
# Cross Entropy

# before:
# logit_maxes = logits.max(1, keepdim=True).values
# norm_logits = logits - logit_maxes # subtract max for numerical stability
# counts = norm_logits.exp()
# counts_sum = counts.sum(1, keepdims=True)
# counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
# probs = counts * counts_sum_inv
# logprobs = probs.log()
# loss = -logprobs[range(n), Yb].mean()

# now:
loss_fast = F.cross_entropy(logits, Yb)
print(loss_fast.item(), 'diff:', (loss_fast - loss).item())

3.885904550552368 diff: -2.384185791015625e-07


Time to calculate the derivatives on a single step!

In [56]:
# Backward pass (of loss)

dlogits = F.softmax(logits, 1)
dlogits[range(batch_size), Yb] -= 1
dlogits /= batch_size

cmp('logits', dlogits, logits) 

logits          | exact: False | approximate: True  | maxdiff: 4.190951585769653e-09


# Exercise 3
that is not necessary to do backprop! Here we will do backprop directly on batch normalization instead of on its small pieces.

In [61]:
# Batch Normalization

# before:
# bnmeani = 1/n*hprebn.sum(0, keepdim=True)
# bndiff = hprebn - bnmeani
# bndiff2 = bndiff**2
# bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n)
# bnvar_inv = (bnvar + 1e-5)**-0.5
# bnraw = bndiff * bnvar_inv
# hpreact = bngain * bnraw + bnbias

# now:
hpreact_fast = bngain * (hprebn - hprebn.mean(0, keepdim=True)) / torch.sqrt(hprebn.var(0, keepdim=True, unbiased=True) + 1e-5) + bnbias
print('max diff:', (hpreact_fast - hpreact).abs().max())

max diff: tensor(4.7684e-07, grad_fn=<MaxBackward1>)


In [64]:
# before we had:
# dbnraw = bngain * dhpreact
# dbndiff = bnvar_inv * dbnraw
# dbnvar_inv = (bndiff * dbnraw).sum(0, keepdim=True)
# dbnvar = (-0.5*(bnvar + 1e-5)**-1.5) * dbnvar_inv
# dbndiff2 = (1.0/(n-1))*torch.ones_like(bndiff2) * dbnvar
# dbndiff += (2*bndiff) * dbndiff2
# dhprebn = dbndiff.clone()
# dbnmeani = (-dbndiff).sum(0)
# dhprebn += 1.0/n * (torch.ones_like(hprebn) * dbnmeani)

# calculate dhprebn given dhpreact (i.e. backprop through the batchnorm)
# (you'll also need to use some of the variables from the forward pass up above)

dhprebn = bngain*bnvar_inv/batch_size * (batch_size*dhpreact - dhpreact.sum(0) - batch_size/(batch_size-1)*bnraw*(dhpreact*bnraw).sum(0))

cmp('hprebn', dhprebn, hprebn) # I can only get approximate to be true, my maxdiff is 9e-10

hprebn          | exact: False | approximate: True  | maxdiff: 1.3969838619232178e-09


# Exercise 4
Finally we will train the MLP neural net with your own backward pass!

### Network

In [66]:
n_embd = 10
n_hidden = 200
block_size = 3
vocab_size = len(chr_to_idx)

# Lookup table
C = torch.randn([vocab_size, n_embd])

# Weights and biases
## Layer 1
W1 = torch.randn([n_embd * block_size, n_hidden]) * (5/3)/((n_embd * block_size) ** 0.5)
b1 = torch.randn([n_hidden]) * 0.1

## Layer 2
W2 = torch.randn([n_hidden, vocab_size]) * 0.1
b2 = torch.randn([vocab_size]) * 0.1

## Batch Norm Shifting weights
bngain = torch.ones((1, n_hidden)) * 0.1 + 1
bnbias = torch.zeros((1, n_hidden)) * 0.1

# Placing parameters in list
parameters = [C, W1, b1, W2, b2, bngain, bnbias]

# Enabling gradients
for p in parameters:
    p.requires_grad = True
print("Number of parameters: " + str(sum(p.nelement() for p in parameters)))

Number of parameters: 12297


### Training

In [70]:
max_steps = 200000
batch_size = 32
lossi = []

for i in range(max_steps):

    # Minibatch
    ix = torch.randint(0, Xtr.shape[0], (batch_size,))
    Xb, Yb = Xtr[ix], Ytr[ix]

    # Forward pass
    emb = C[Xb]
    embcat = emb.view(emb.shape[0], -1)
    
    ## Linear layer
    hprebn = embcat @ W1 + b1 # hidden layer pre-activation
    
    ## BatchNorm layer
    bnmean = hprebn.mean(0, keepdim=True)
    bnvar = hprebn.var(0, keepdim=True, unbiased=True)
    bnvar_inv = (bnvar + 1e-5)**-0.5
    bnraw = (hprebn - bnmean) * bnvar_inv
    hpreact = bngain * bnraw + bnbias
    
    ## Non-linearity
    h = torch.tanh(hpreact) # hidden layer
    logits = h @ W2 + b2 # output layer
    loss = F.cross_entropy(logits, Yb) # loss function

    # Backward pass
    for p in parameters:
      p.grad = None
    
    # loss.backward() # use this for correctness comparisons
    
    ## Manual backprop!
    ## Logits
    dlogits = F.softmax(logits, 1)
    dlogits[range(batch_size), Yb] -= 1
    dlogits /= batch_size
    
    ## Layer 2
    dh = dlogits @ W2.T
    dW2 = h.T @ dlogits
    db2 = dlogits.sum(0)
    
    ## Non-linearity
    dhpreact = (1.0 - h**2) * dh
    
    ## BatchNorm layer
    dbngain = (bnraw * dhpreact).sum(0, keepdim=True)
    dbnbias = dhpreact.sum(0, keepdim=True)
    dhprebn = bngain*bnvar_inv/batch_size * (batch_size*dhpreact - dhpreact.sum(0) - batch_size/(batch_size-1)*bnraw*(dhpreact*bnraw).sum(0))
    
    ## Layer 1
    dembcat = dhprebn @ W1.T
    dW1 = embcat.T @ dhprebn
    db1 = dhprebn.sum(0)
    
    ## Embeddings
    demb = dembcat.view(emb.shape)
    dC = torch.zeros_like(C)
    for k in range(Xb.shape[0]):
      for j in range(Xb.shape[1]):
        ix = Xb[k,j]
        dC[ix] += demb[k,j]
    grads = [dC, dW1, db1, dW2, db2, dbngain, dbnbias]

    # Update
    lr = 0.1 if i < 100000 else 0.01
    for p, grad in zip(parameters, grads):
      p.data += -lr * grad

    # Track stats
    if i % 10000 == 0: # print every once in a while
      print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
    lossi.append(loss.log10().item())

      0/ 200000: 3.5072
  10000/ 200000: 2.4028
  20000/ 200000: 2.2744
  30000/ 200000: 1.8583
  40000/ 200000: 2.6295
  50000/ 200000: 2.2820
  60000/ 200000: 1.9982
  70000/ 200000: 1.9411
  80000/ 200000: 2.2767
  90000/ 200000: 2.4410
 100000/ 200000: 2.5345
 110000/ 200000: 2.2544
 120000/ 200000: 2.3097
 130000/ 200000: 2.1966
 140000/ 200000: 1.9865
 150000/ 200000: 2.2198
 160000/ 200000: 2.0367
 170000/ 200000: 2.0724
 180000/ 200000: 1.8182
 190000/ 200000: 2.0426


### Results

In [72]:
# Calibrate the batch norm at the end of training
with torch.no_grad():
    # pass the training set through
    emb = C[Xtr]
    embcat = emb.view(emb.shape[0], -1)
    hpreact = embcat @ W1 + b1
    # measure the mean/std over the entire training set
    bnmean = hpreact.mean(0, keepdim=True)
    bnvar = hpreact.var(0, keepdim=True, unbiased=True)

In [75]:
# Evaluating train and val loss
@torch.no_grad()
def split_loss(split):
    x,y = {
        'train': (Xtr, Ytr),
        'valid': (Xval, Yval),
        'test': (Xtest, Ytest),
    }[split]
    emb = C[x] # (N, block_size, n_embd)
    embcat = emb.view(emb.shape[0], -1) # concat into (N, block_size * n_embd)
    hpreact = embcat @ W1 + b1
    hpreact = bngain * (hpreact - bnmean) * (bnvar + 1e-5)**-0.5 + bnbias
    h = torch.tanh(hpreact) # (N, n_hidden)
    logits = h @ W2 + b2 # (N, vocab_size)
    loss = F.cross_entropy(logits, y)
    print(split, loss.item())

split_loss('train')
split_loss('valid')

train 2.066371440887451
valid 2.114915609359741


In [78]:
# Sample from model
for _ in range(20):
    out = []
    context = [0] * block_size # initialize with all ...
    while True:
        # forward pass:
        # Embedding
        emb = C[torch.tensor([context])] # (1,block_size,d)      
        embcat = emb.view(emb.shape[0], -1) # concat into (N, block_size * n_embd)
        hpreact = embcat @ W1 + b1
        hpreact = bngain * (hpreact - bnmean) * (bnvar + 1e-5)**-0.5 + bnbias
        h = torch.tanh(hpreact) # (N, n_hidden)
        logits = h @ W2 + b2 # (N, vocab_size)
        
        # Sample
        probs = F.softmax(logits, dim=1)
        ix = torch.multinomial(probs, num_samples=1).item()
        context = context[1:] + [ix]
        out.append(ix)
        if ix == 0:
            break
    
    print(''.join(idx_to_chr[i] for i in out))

aldie.
nevy.
tree.
clivette.
arri.
robee.
khalegeccyla.
solaci.
shawn.
zatleigh.
yoluwa.
avagan.
dani.
janze.
moegifven.
katelan.
brylintson.
bravedaanna.
via.
karynn.
